In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import sys
sys.path.append("..")  # add top folder to path

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import moses
import gentrl

import tinymolecule as tiny

## Initializing the encoder multilayer perceptron (MLP) and decoder MLP to form the variational autoencoder
We note that the latent size for decoder is twice as small because we're drawing a sample from a distribution, whereas for the encoder, we get two values: mean and log variance. The MLPs are relatively small neural networks, by default there are two hidden layers, each with 100 neurons, and output layer (latent space) is 10 neurons. The decoder shares a similar structure but in reverse: the input from latent space is 5 neurons, with two hidden layers of 100 neurons.

In [78]:
enc = tiny.LinearEncoder(latent_size=10)  # mu and sigma
dec = tiny.LinearDecoder(latent_size=5)  # a single sample
vae = tiny.TinyVAE(enc, dec)

## Loading in assay data on CCR4
In the future, we will filter this data based on an IC50 threshold to pick out only relevant molecules.

In [3]:
data_dir = "/Users/Munchic/Developer/Capstone/tinymolecule/data/ccr4_ic50_meta.csv"
assays = pd.read_csv(data_dir)
assays.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,canonical_smiles,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,NaN,31863,[],CHEMBL663853,Inhibitory concentration against human DNA top...,B,BAO_0000190,BAO_0000357,single protein format,c1ccc(-c2nc3c(-c4nc5ccccc5o4)cccc3o2)cc1,...,Homo sapiens,DNA topoisomerase II alpha,9606.0,NaN,NaN,IC50,uM,UO_0000065,NaN,100.0
1,NaN,31864,[],CHEMBL872937,In vivo inhibitory activity against human Hepa...,B,BAO_0000190,BAO_0000218,organism-based format,Cc1ccc2oc(-c3cccc(N4C(=O)c5ccc(C(=O)O)cc5C4=O)...,...,Homo sapiens,Heparanase,9606.0,NaN,NaN,IC50,uM,UO_0000065,NaN,2.5
2,NaN,31865,[],CHEMBL693237,In vivo concentration required against angioge...,F,BAO_0000190,BAO_0000218,organism-based format,Cc1ccc2oc(-c3cccc(N4C(=O)c5ccc(C(=O)O)cc5C4=O)...,...,NaN,NON-PROTEIN TARGET,NaN,NaN,NaN,IC50,uM,UO_0000065,NaN,50.0
3,NaN,31866,[],CHEMBL872937,In vivo inhibitory activity against human Hepa...,B,BAO_0000190,BAO_0000218,organism-based format,COc1ccccc1-c1ccc2oc(-c3ccc(OC)c(N4C(=O)c5ccc(C...,...,Homo sapiens,Heparanase,9606.0,NaN,NaN,IC50,uM,UO_0000065,NaN,9.0
4,Not Determined,31867,[],CHEMBL693238,In vivo concentration required against angioge...,F,BAO_0000190,BAO_0000218,organism-based format,COc1ccccc1-c1ccc2oc(-c3ccc(OC)c(N4C(=O)c5ccc(C...,...,NaN,NON-PROTEIN TARGET,NaN,NaN,NaN,IC50,uM,NaN,NaN,NaN


In [57]:
assays_no_cx_chir = assays[assays["canonical_smiles"].str.contains("@|I|[\+]|\-|\.|\/|Se|P|7|8") == False] # get assays involving molecules without complex chiral centers since gentrl cannot encode them
assays_no_cx_chir.head()

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,canonical_smiles,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
17,NaN,31880,[],CHEMBL651396,Effective concentration of compound achieving ...,F,BAO_0000188,BAO_0000219,cell-based format,Cc1cn(C2C=CC(COC(=O)CN3CCNCC3)O2)c(=O)[nH]c1=O,...,Homo sapiens,CCRF-CEM,9606.0,NaN,NaN,EC50,uM,UO_0000065,NaN,0.0636
18,NaN,31881,[],CHEMBL657398,Cytotoxic concentration of compound required t...,F,BAO_0000187,BAO_0000219,cell-based format,Cc1cn(C2C=CC(COC(=O)CN3CCNCC3)O2)c(=O)[nH]c1=O,...,Homo sapiens,CCRF-CEM,9606.0,NaN,NaN,CC50,uM,UO_0000065,NaN,1000.0000
19,NaN,31882,[],CHEMBL845142,Selectivity index expressed as the ratio of CC...,F,BAO_0000179,BAO_0000019,assay format,Cc1cn(C2C=CC(COC(=O)CN3CCNCC3)O2)c(=O)[nH]c1=O,...,NaN,Unchecked,NaN,NaN,NaN,Selectivity index,uM,UO_0000064,NaN,15723.0000
20,Not Determined,31883,[],CHEMBL752464,Antimycobacterial activity against Mycobacteri...,F,BAO_0000376,BAO_0000218,organism-based format,Cc1cn(C2C=CC(COC(=O)CN3CCNCC3)O2)c(=O)[nH]c1=O,...,Mycobacterium tuberculosis,Mycobacterium tuberculosis,1773.0,NaN,NaN,Inhibition,NaN,NaN,NaN,NaN
21,NaN,31884,[],CHEMBL618375,In vitro hydrolysis in human plasma,A,BAO_0002115,BAO_0000366,cell-free format,Cc1cn(C2C=CC(COC(=O)CN3CCNCC3)O2)c(=O)[nH]c1=O,...,NaN,ADMET,NaN,NaN,NaN,T1/2,min,UO_0000032,NaN,20.0000


In [61]:
molecules = gentrl.tokenizer.encode(assays_no_cx_chir["canonical_smiles"])[0].float()  # encode SMIl
molecules[0]  # an example molecule

tensor([ 1., 23., 10., 24., 10., 18., 22., 23.,  3., 23., 13., 23., 23., 22.,
        23.,  9., 23., 22., 13.,  9., 15., 23.,  6., 14., 23., 23.,  6., 23.,
        23., 14., 15.,  9.,  3., 15., 10., 22., 13.,  9., 15.,  7., 18., 21.,
        11., 10., 24., 13.,  9.,  2.,  2.,  2.])

## Encoder time!
Let's see how our encoder works. When we pass in this 50-dimensional vector, it will spit out a 10-dimension vector, 5 dimensions to encode a mean, 5 for standard deviation in variational autoencoder's latent space. Essentially, each molecule becomes a distribution, then it's reconstruction is a sample from this distribution.

In [70]:
out = vae(molecules[0:1])
print("reconstructred molecule (untrained VAE):\n", out[0].detach(), "\n")
print("encoded mean (untrained VAE):\n", out[1].detach())
print("encoded standard deviation (untrained VAE):\n", out[2].detach())

reconstructred molecule (untrained VAE):
 tensor([[0.9150, 0.1977, 0.4159, 0.1884, 0.1550, 0.6221, 0.8697, 0.1161, 0.4034,
         0.2778, 0.7257, 0.8384, 0.4224, 0.3813, 0.4244, 0.1259, 0.8496, 0.7491,
         0.2795, 0.1357, 0.4845, 0.2802, 0.7064, 0.7032, 0.8369, 0.3409, 0.8517,
         0.7884, 0.6800, 0.1182, 0.9210, 0.2303, 0.5316, 0.8936, 0.5819, 0.4934,
         0.5213, 0.3806, 0.1930, 0.2155, 0.4617, 0.5326, 0.8857, 0.8150, 0.7056,
         0.3036, 0.3421, 0.5146, 0.1296, 0.3390]]) 

encoded mean (untrained VAE):
 tensor([[-2.5202, -4.7714, 10.0101, -3.4116, -8.4753]])
encoded standard deviation (untrained VAE):
 tensor([[-7.3000,  2.4417, -0.1557,  1.7446, -4.5686]])


In [71]:
# it should not make any sense as we try to convert this failed reconstructed vector into a SMILES string
print("Reconstructed SMILES (there are not even atoms here):", gentrl.tokenizer.decode(out[0].detach()))

Reconstructed SMILES (there are not even atoms here): ['']


## Time to train the VAE


In [74]:
molecloader = tiny.dataset.molecloader(molecules)
optimizer = optim.Adam(vae.parameters(), lr=1e-2)
criterion = nn.BCELoss(reduction='sum')

In [81]:
train_loss = tiny.train.simple_train(vae, molecloader, criterion, optimizer)

epoch 1/5 started at 0.0000 s
epoch_loss: inf
epoch 2/5 started at 0.5721 s
epoch_loss: inf
epoch 3/5 started at 1.1109 s
epoch_loss: inf
epoch 4/5 started at 1.6407 s
epoch_loss: 5.204864564569053e+30
epoch 5/5 started at 2.1745 s
epoch_loss: 1.2436008182901558e+35


## Results of a trained VAE

In [82]:
# let's try on one molecule
print("actual molecule:\n", molecules[0:1])
print("actual SMILES:", gentrl.tokenizer.decode(molecules[0:1]), "\n")
print("reconstructed molecule:\n", vae(molecules[0:1])[0].detach())
print("reconstructed SMILES:", gentrl.tokenizer.decode(vae(molecules[0:1])[0].detach()), "\n")

actual molecule:
 tensor([[ 1., 23., 10., 24., 10., 18., 22., 23.,  3., 23., 13., 23., 23., 22.,
         23.,  9., 23., 22., 13.,  9., 15., 23.,  6., 14., 23., 23.,  6., 23.,
         23., 14., 15.,  9.,  3., 15., 10., 22., 13.,  9., 15.,  7., 18., 21.,
         11., 10., 24., 13.,  9.,  2.,  2.,  2.]])
actual SMILES: ['Cc1cn(C2C=CC(COC(=O)CN3CCNCC3)O2)c(=O)[nH]c1=O'] 

reconstructed molecule:
 tensor([[4.1921e-04, 9.9724e-01, 7.5411e-01, 1.0000e+00, 9.9646e-01, 7.0786e-08,
         3.0109e-04, 6.0437e-01, 1.2025e-05, 3.7876e-03, 6.7907e-06, 9.8201e-01,
         2.7855e-03, 9.9838e-01, 4.8828e-01, 2.7778e-01, 9.9988e-01, 6.7359e-01,
         9.9935e-01, 9.9959e-01, 9.9862e-01, 8.2921e-01, 1.2318e-05, 1.0000e+00,
         1.5602e-02, 3.2284e-04, 2.9124e-06, 9.9990e-01, 4.8633e-04, 1.0000e+00,
         7.2253e-05, 3.3235e-04, 1.0842e-01, 9.6054e-01, 3.8089e-01, 7.9761e-03,
         3.9960e-06, 3.3483e-02, 1.1743e-06, 9.9998e-01, 9.9908e-01, 3.2052e-03,
         9.9999e-01, 7.2967e-02, 9

## What's wrong
I ran the reconstructed tensor through a sigmoid function to use for binary cross entropy but it's not comparable to the tensors of natural numbers from the original samples. I should be using evidence lower bound (ELBO) loss.